In [1]:
from processData import MLDataAnalyzer
import pandas as pd
from generateConfig import GenerateConfigLLM
from trainingSchool import TrainingSchool 

2025-01-10 15:10:07.662202: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-10 15:10:07.694542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736502007.716334  172210 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736502007.723199  172210 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-10 15:10:07.744810: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
df = pd.read_csv("example_data.csv")

In [3]:


task_description = "Using the dataframe I need a binary classification algorithm. This is a sequence data"

models = [
    'StandardScaler',
    'MinMaxScaler',
    'RobustScaler',
    'LinearRegression',
    'Ridge',
    'RandomForestRegressor',
    'SVR',
    'XGBRegressor',
    'MLPRegressor',
    'LogisticRegression',
    'RandomForestClassifier',
    'SVC',
    'XGBClassifier',
    'MLPClassifier'
]


analyzer = MLDataAnalyzer(df, "target", task_description)
sample_data = analyzer.get_sample_data()
analysis = analyzer.analyze_dataset()
config_generator = GenerateConfigLLM()
config = config_generator.generate_config(analysis, sample_data, task_description, models)
print(config)

scalers:
  StandardScaler:
    name: StandardScaler
    params: {}

preprocessing:
  missing_values:
    strategy: ignore
  categorical:
    strategy: ignore
  feature_selection:
    enabled: false
    method: recursive_feature_elimination
    params: {}

models:
  classification:
    LogisticRegression:
      name: LogisticRegression
      params: {}

deep_learning:
  lstm:
    enabled: true
    params:
      units: [64, 32]
      dropout_rates: [0.2, 0.2]
      optimizer: adam
      batch_size: 32
      epochs: 100
  
  transformer:
    enabled: true
    params:
      num_heads: 8
      key_dim: 64
      dropout_rate: 0.2
      dense_units: [64, 32]
      optimizer: adam
      batch_size: 32
      epochs: 100

evaluation:
  cv_folds: 5
  stratify: false
  metrics:
    classification: ['accuracy', 'f1', 'precision', 'recall']

sequence_data:
  enabled: true


In [4]:
import pickle
import os

def save_model(model, filepath):
    """
    Save a trained model to a file using pickle.
    
    Parameters:
    model: sklearn model object
        The trained model to save
    filepath: str
        Path where the model will be saved
    """
    try:
        with open(filepath, 'wb') as file:
            pickle.dump(model, file)
        print(f"Model successfully saved to {filepath}")
    except Exception as e:
        print(f"Error saving model: {str(e)}")

def load_model(filepath):
    """
    Load a model from a pickle file.
    
    Parameters:
    filepath: str
        Path to the saved model file
    
    Returns:
    The loaded model object
    """
    try:
        with open(filepath, 'rb') as file:
            model = pickle.load(file)
        print(f"Model successfully loaded from {filepath}")
        return model
    except Exception as e:
        print(f"Error loading model: {str(e)}")
        return None

In [5]:
yaml_content = config.strip().strip('`').lstrip('yml').strip()


In [6]:
 trainer = TrainingSchool(config_path="config1.yaml")

# Assuming X and y are your data
# For demonstration, let's create dummy data:
X = df.drop("target", axis=1).values
y = df["target"].values


# Fit the model
trainer.fit(X, y)

# Retrieve and print best model details
best_info = trainer.get_best_model()
print("Best Model Info:", best_info)

No GPU found. Running on CPU.
Detected task type: classification
Score for logistic_regression with standard scaler: 0.9800
Score for LSTM with standard scaler: 0.9410
Score for Transformer with standard scaler: 0.5250

Best model: logistic_regression
Best scaler: StandardScaler
Best score: 0.9800
Best Model Info: {'model': LogisticRegression(), 'scaler': StandardScaler(), 'score': 0.9800000000000001, 'model_name': 'logistic_regression'}


In [25]:
import tempfile
import os

filename = "example.csv"

with tempfile.NamedTemporaryFile(mode='w+', suffix='.yaml', delete=False) as temp_config_file:
    temp_config_file.write(yaml_content)
    temp_config_file.flush()
    config_path = temp_config_file.name
    
    trainer = TrainingSchool(config_path=config_path)
    X = df.drop("target", axis=1).values
    y = df["target"].values
    
    trainer.fit(X, y)
    best_info = trainer.get_best_model()
    
    # Save the actual model using joblib
    model_filename = f"{filename.rsplit('.', 1)[0]}_model.pkl"

    test_df = df.drop("target", axis=1)

    # Test the model
    X_test = test_df[required_features].values
    X_test_scaled = best_info['scaler'].transform(X_test)
    predictions = best_info['model'].predict(X_test_scaled)

    # Create a new DataFrame with predictions
    result_df = test_df.copy()
    result_df['target'] = predictions
    y_test = model.predict(X_test_scaled)

    print("Accuracy of the model:", accuracy_score(y, y_test))

    save_model(best_info['model'], model_filename)
    
    loaded_model = load_model(model_filename)

    X_test_scaled = best_info['scaler'].transform(X_test)
    predictions = best_info['model'].predict(X_test_scaled)


    # Create a new DataFrame with predictions
    result_df = test_df.copy()
    result_df['target'] = predictions
    y_test = loaded_model.predict(X_test_scaled)




    
    # Create serializable model info
    model_info = {
        'filename': filename,
        'task_description': task_description,
        'best_model': {
            'model_type': type(best_info['model']).__name__,
            'score': best_info.get('score', None),
            'model_path': model_filename  # Save the path to the saved model
        },
        'feature_columns': list(df.drop("target", axis=1).columns)
    }
    
    print(model_info)
    
    os.unlink(config_path)
    
  

No GPU found. Running on CPU.
Detected task type: classification
Score for LogisticRegression with StandardScaler scaler: 0.9800
Score for LSTM with StandardScaler scaler: 0.9355
Score for Transformer with StandardScaler scaler: 0.5250

Best model: LogisticRegression
Best scaler: StandardScaler
Best score: 0.9800
Accuracy of the model: 0.99
Model successfully saved to example_model.pkl
Model successfully loaded from example_model.pkl
{'filename': 'example.csv', 'task_description': 'Using the dataframe I need a binary classification algorithm. This is a sequence data', 'best_model': {'model_type': 'LogisticRegression', 'score': 0.9800000000000001, 'model_path': 'example_model.pkl'}, 'feature_columns': ['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4']}


In [21]:


test_df = df.drop("target", axis=1)

# X.to_csv("test_data.csv", index=False)

In [12]:


required_features = model_info['feature_columns']
if not all(feature in test_df.columns for feature in required_features):
    print(jsonify({'error': 'Test data missing required features'}))

model_path = "example_model.pkl"
model = load_model(model_path)
# Make predictions using the model


Model successfully loaded from example_model.pkl


In [13]:
from sklearn.metrics import accuracy_score

In [14]:
X_test = test_df[required_features].values
predictions = model.predict(X_test)

# Create a new DataFrame with predictions
result_df = test_df.copy()
result_df['target'] = predictions

In [15]:
result_df

,feature_0,feature_1,feature_2,feature_3,feature_4,target
0,0.374540,0.950714,0.731994,0.598658,0.156019,1
1,0.155995,0.058084,0.866176,0.601115,0.708073,1
2,0.020584,0.969910,0.832443,0.212339,0.181825,1
3,0.183405,0.304242,0.524756,0.431945,0.291229,1
4,0.611853,0.139494,0.292145,0.366362,0.456070,1
...,...,...,...,...,...,...
195,0.262264,0.595078,0.051426,0.496366,0.596843,1
196,0.334244,0.770912,0.106598,0.075138,0.728189,1
197,0.495491,0.688402,0.434827,0.246402,0.819102,1
198,0.799416,0.694696,0.272145,0.590231,0.360974,1


In [16]:
result_df.target.unique()

array([1])

In [17]:
save_model(model, "test_model")

Model successfully saved to test_model


In [18]:
loaded_model = load_model("test_model")

Model successfully loaded from test_model


In [19]:
y_test = loaded_model.predict(X_test)

accuracy_score(y, y_test)

0.475